In [ ]:
import json
import copy
import random
import time

#Functions for generating game states
def generate_next_gameloop(current_state, current_round):
    """
    Generates the next game loop in a game based on the current state.

    :param current_state: A JSON object representing the current state of the game.
    :return: A JSON object representing the next state of the game.
    """
    # Deep copy the current state to avoid modifying the original state
    new_state = copy.deepcopy(current_state)
    # Increment the game loop number
    new_state[f"loop_{current_round + 1}"] = copy.deepcopy(current_state[f"loop_{current_round}"])

    # Process each bike and its agents
    for bike in new_state[f"loop_{current_round + 1}"]['bikes'].values():
        # Check if the awdi intersects this bike
        awdi_intersects = new_state[f"loop_{current_round + 1}"]['awdi']['target'] == bike['id']
        bike["position"]["x"] = bike["position"]["x"] + random.randint(-10, 10)
        bike["position"]["y"] = bike["position"]["y"] + random.randint(-10, 10)
        # Process each agent on the bike
        for agent in bike['agents'].values():
            # If the awdi intersects, remove all agents from the bike
            if awdi_intersects:
                agent['energy'] = 0  # Setting energy to 0 as they are eliminated

            # Update agent's energy and check if they should be moved to bike 0
            if agent['energy'] > 0:
                agent['energy'] -= 1  # Decrease energy by 1 for this loop
                if agent['energy'] == 0:
                    # Move to bike 0
                    agent['bikeID'] = '0x0'
            else:
                # Already on bike 0 or moved to bike 0
                agent['bikeID'] = '0x0'

    # Handle agents on bike 0 (this can be expanded with more logic if needed)
    # Currently, it only ensures they are on bike 0
    # update awdi position
    new_state[f"loop_{current_round + 1}"]['awdi']['position']['x'] += random.randint(-10, 10)
    new_state[f"loop_{current_round + 1}"]['awdi']['position']['y'] += random.randint(-10, 10)
    return new_state

def generate_initial_state():
    """
    Generates an initial game state with a random number of bikes and agents.

    :return: A JSON object representing the initial state of the game.
    """
    # Define possible colors for agents
    possible_colours = ["Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "White"]

    # Initialize the game state
    initial_state = {
        "loop_0": {
            "bikes": {},
            "lootboxes": {},  # This can be populated with lootbox data if needed
            "awdi": {
                "target": None,  # No target initially
                "position": {"x": random.randint(-300, 300), "y": random.randint(-300, 300)},
            }
        },
    }

    # Randomly determine the number of bikes
    num_bikes = random.randint(6, 10)

    for bike_id in range(1, num_bikes + 1):
        # Each bike will have a unique ID
        bike_key = f"bike_{bike_id}"

        # Randomly determine the number of agents on this bike
        num_agents = random.randint(3, 9)
        bike_agents = {}

        for agent_id in range(1, num_agents + 1):
            # Each agent will have a unique ID and randomly assigned color
            agent_key = f"agent_{bike_id}_{agent_id}"
            agent_colour = random.choice(possible_colours)

            # Define the agent
            bike_agents[agent_key] = {
                "id": agent_key,
                "groupID": random.randint(0, 8),
                "colour": agent_colour,
                "points": 0,
                "energy": 100,  # Initial energy can be adjusted
                # The rest of the agent's attributes are placeholders
                "bikes": {},
                "forces": {},
                "lootboxChoice": None,
                "lootDistribution": [],
            }

        # Add the bike and its agents to the game state
        initial_state["loop_0"]["bikes"][bike_key] = {
            "id": bike_key,
            "position": {"x": random.randint(-300, 300), "y": random.randint(-300, 300)},
            "agents": bike_agents
        }

    # add lootboxes
    num_lootboxes = random.randint(4, 12)
    for lootbox_id in range(1, num_lootboxes + 1):
        lootbox_key = f"lootbox_{lootbox_id}"
        initial_state["loop_0"]["lootboxes"][lootbox_key] = {
            "id": lootbox_key,
            "position": {
                "x": random.randint(-500, 500),
                "y": random.randint(-500, 500)
            },
            "colour": random.choice(possible_colours),
        }
    
    # add awdi
    initial_state["loop_0"]["awdi"]["id"] = "awdi"
    initial_state["loop_0"]["awdi"]["position"] = {
        "x": random.randint(-500, 500),
        "y": random.randint(-500, 500)
    }

    return initial_state


In [1]:
# Generate the initial state and run the game for LOOPS number of loops
initial_state = generate_initial_state()
LOOPS = 10
for i in range(LOOPS):
    initial_state = generate_next_gameloop(initial_state, i)
with open(f"./visualiser/JSON/test2.json", "w") as f:
    json.dump(initial_state, f, indent=4)

NameError: name 'generate_initial_state' is not defined

In [8]:
import json
with open("./../game_dump.json", "r") as f:
    data = json.load(f)

In [ ]:
for bike in data[0]["bikes"]:
    print(bike["agents"][0])

In [13]:
print(bike)

{'id': 'c3f60ba6-9917-4af2-854c-dddda481f1a5', 'physical_state': {'position': {'x': 379.2924977174136, 'y': 18.741400417727487}, 'acceleration': 0, 'velocity': 0, 'mass': 50}, 'agents': []}


In [ ]:
# for audi in data[0]["audi":]
print(data[0]["audi"])

In [14]:
print(data[0]["loot_boxes"][0])

{'id': 'ab3ae98d-cc77-49e7-a61a-12f1e192adbd', 'physical_state': {'position': {'x': 174.19206318464458, 'y': 401.2478960094951}, 'acceleration': 0, 'velocity': 0, 'mass': 0}, 'total_resources': 6.916819697775168, 'colour': 'orange'}


In [11]:
print(data[0])
len(data)

[{'iteration': -1, 'agents': {'0f2045aa-9c8a-4c5d-a287-344146f0051c': {'class': 'objects.BaseBiker', 'forces': {'pedal': 0, 'brake': 0, 'turning': {'steer_bike': False, 'steering_force': 0}}, 'energy_level': 1, 'points': 0, 'colour': 'white', 'location': {'x': 42.44123631944164, 'y': 53.969399424823166}, 'on_bike': True, 'bike_id': 'fa60449c-426c-4fc0-9530-bb5cb570df95', 'reputation': None, 'group_id': 0}, '30e9c4a4-a5cf-482b-b1f4-3a76610f309f': {'class': 'objects.BaseBiker', 'forces': {'pedal': 0, 'brake': 0, 'turning': {'steer_bike': False, 'steering_force': 0}}, 'energy_level': 1, 'points': 0, 'colour': 'pink', 'location': {'x': 42.44123631944164, 'y': 53.969399424823166}, 'on_bike': True, 'bike_id': 'fa60449c-426c-4fc0-9530-bb5cb570df95', 'reputation': None, 'group_id': 0}, '61c01722-ee9b-4678-908f-a22c206a2b1e': {'class': 'team1.Biker1', 'forces': {'pedal': 0, 'brake': 0, 'turning': {'steer_bike': False, 'steering_force': 0}}, 'energy_level': 1, 'points': 0, 'colour': 'blue', 'loc

10

In [9]:
import csv
import os

def calculate_low_frame_rate(frame_rates, percentile):
    # Sort frame rates in ascending order
    sorted_frame_rates = sorted(frame_rates)

    # Calculate the index for the percentile
    index = int(len(sorted_frame_rates) * percentile)
    return sorted_frame_rates[index]

# Prepare a list to store the results
results = []

# Process every file in the folder
for file in os.listdir("./profiles/"):
    if file.endswith(".csv"):
        with open(f"./profiles/{file}", "r") as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header

            frame_rates = []
            for row in reader:
                if len(row) > 1:  # Assuming the frame rate is the second column
                    try:
                        frame_rate = float(row[1])
                        frame_rates.append(frame_rate)
                    except ValueError:
                        pass  # Ignore rows where conversion to float fails

            if frame_rates:
                avg_frame_rate = sum(frame_rates) / len(frame_rates)
                one_percent_low = calculate_low_frame_rate(frame_rates, 0.01)
                zero_point_one_percent_low = calculate_low_frame_rate(frame_rates, 0.001)
                results.append({
                    "File": file,
                    "Average FPS": avg_frame_rate,
                    "1% Low FPS": one_percent_low,
                    "0.1% Low FPS": zero_point_one_percent_low
                })

                print(f"File: {file}")
                print(f"Average Frame Rate: {avg_frame_rate} FPS")
                print(f"1% Low Frame Rate: {one_percent_low} FPS")
                print(f"0.1% Low Frame Rate: {zero_point_one_percent_low} FPS\n")
            else:
                print(f"File: {file} contains no valid frame rate data.")


File: fpsnew_10.csv
Average Frame Rate: 173.14358625313827 FPS
1% Low Frame Rate: 105.2631607055664 FPS
0.1% Low Frame Rate: 70.92198181152344 FPS

File: fpsnew_20.csv
Average Frame Rate: 124.71520802268925 FPS
1% Low Frame Rate: 57.803470611572266 FPS
0.1% Low Frame Rate: 30.674846649169922 FPS

File: fpsnew_40.csv
Average Frame Rate: 93.81230645796832 FPS
1% Low Frame Rate: 5.685048580169678 FPS
0.1% Low Frame Rate: 5.685048580169678 FPS

File: fpsnew_6.csv
Average Frame Rate: 198.3039180647816 FPS
1% Low Frame Rate: 116.27906799316406 FPS
0.1% Low Frame Rate: 82.6446304321289 FPS

File: fpsnew_60.csv
Average Frame Rate: 58.820847451395395 FPS
1% Low Frame Rate: 1.3987970352172852 FPS
0.1% Low Frame Rate: 0.9111617207527161 FPS

File: fpsnew_8.csv
Average Frame Rate: 193.45315265655518 FPS
1% Low Frame Rate: 119.04762268066406 FPS
0.1% Low Frame Rate: 86.20690155029297 FPS

File: fpsnew_80.csv
Average Frame Rate: 55.65018015278606 FPS
1% Low Frame Rate: 3.47947096824646 FPS
0.1% Low 

In [ ]:
# Write the results to a CSV file
output_file = "./profiles/fpsres/results.csv"
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ["File", "Average FPS", "1% Low FPS", "0.1% Low FPS"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)